<a href="https://colab.research.google.com/github/kawasaki-koto/only_colab_notebook/blob/main/Dataset_Maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **このノートブックは私が改良し、機能を追加したものです**
（オリジナルは下に）

# 📊 Dataset Maker by Hollowstrawberry

This is based on the work of [Kohya-ss](https://github.com/kohya-ss/sd-scripts) and [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb). Thank you!

### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning inference.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Homepage** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |

In [ ]:
import os
from IPython import get_ipython
from IPython.display import display, Markdown

COLAB = True

if COLAB:
  from google.colab.output import clear as clear_output
else:
  from IPython.display import clear_output

#@title ## 🚩 Start Here

#@markdown ### 1️⃣ Setup
#@markdown This cell will load some requirements and create the necessary folders in your Google Drive. <p>
#@markdown Your project name can't contain spaces but it can contain a single / to make a subfolder in your dataset. <p>https://huggingface.co/motetetet/Motet_Lora
project_name = "KirisameMarisaExplicit1210" #@param {type:"string"}
project_name = project_name.strip()
#@markdown The folder structure doesn't matter and is purely for comfort. Make sure to always pick the same one. I like organizing by project.
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)" #@param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]

if not project_name or any(c in project_name for c in " .()\"'\\") or project_name.count("/") > 1:
  print("Please write a valid project_name.")
else:
  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  project_base = project_name if "/" not in project_name else project_name[:project_name.rfind("/")]
  project_subfolder = project_name if "/" not in project_name else project_name[project_name.rfind("/")+1:]

  root_dir = "/content" if COLAB else "~/Loras"
  deps_dir = os.path.join(root_dir, "deps")

  if "/Loras" in folder_structure:
    main_dir      = os.path.join(root_dir, "drive/MyDrive/StableDiffusion/Loras") if COLAB else root_dir
    config_folder = os.path.join(main_dir, project_base)
    images_folder = os.path.join(main_dir, project_base, "dataset")
    if "/" in project_name:
      images_folder = os.path.join(images_folder, project_subfolder)
  else:
    main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
    config_folder = os.path.join(main_dir, "config", project_name)
    images_folder = os.path.join(main_dir, "datasets", project_name)

  for dir in [main_dir, deps_dir, images_folder, config_folder]:
    os.makedirs(dir, exist_ok=True)

  print(f"✅ Project {project_name} is ready!")
  step1_installed_flag = True

def ubuntu_deps():
  print("🏭 Installing dependencies...\n")
  !apt -y install aria2
  return not get_ipython().__dict__['user_ns']['_exit_code']

if "step2_installed_flag" not in globals():
  if ubuntu_deps():
    clear_output()
    step2_installed_flag = True
  else:
    print("❌ Error installing dependencies, attempting to continue anyway...")


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

import json
import time
from urllib.request import urlopen, Request

#@title ### 2️⃣ Scrape images from Gelbooru

#@markdown We will grab images from the popular anime gallery [Gelbooru](https://gelbooru.com/). Images are sorted by tags, including poses, scenes, character traits, character names, artists, etc. <p>
#@markdown * If you instead want to use your own images, upload them to your Google Drive's `Loras/project_name/dataset` folder.
#@markdown * If you instead want to download screencaps of anime episodes, try [this other colab by another person](https://colab.research.google.com/drive/1oBSntB40BKzNmKceXUlkXzujzdQw-Ci7). It's more complicated though.

#@markdown Up to 1000 images may be downloaded by this step in just one minute. Don't abuse it. <p>
#@markdown Your target tags should include the relevant tags for your character/concept/artstyle, and exclude undesired tags (for example, explicit images may affect learning).
#@markdown Separate words with underscores, separate tags with spaces, and use - to exclude a tag. You can also include a minimum score: `score:>10`
scrape_tags = "kirisame_marisa rating:explicit -animated" #@param {type:"string"}
##@markdown If an image is bigger than this resolution a smaller version will be downloaded instead.
max_resolution = 3072 #@param {type:"slider", min:1024, max:8196, step:1024}
##@markdown Posts with a parent post are often minor variations of the same image.
include_posts_with_parent = True #param {type:"boolean"}

tags = scrape_tags.replace(" ", "+")\
           .replace("(", "%28")\
           .replace(")", "%29")\
           .replace(":", "%3a")\
           .replace("&", "%26")\

url = "https://gelbooru.com/index.php?page=dapi&json=1&s=post&q=index&limit=100&tags={}".format(tags)
user_agent = "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/93.0.4577.83 Safari/537.36"
limit = 100 # hardcoded by gelbooru
total_limit = 1000 # you can edit this if you want but I wouldn't recommend it
supported_types = (".png", ".jpg", ".jpeg")

def get_json(url):
  with urlopen(Request(url, headers={"User-Agent": user_agent})) as page:
    return json.load(page)

def filter_images(data):
  return [p["file_url"] if p["width"]*p["height"] <= max_resolution**2 else p["sample_url"]
          for p in data["post"]
          if (p["parent_id"] == 0 or include_posts_with_parent)
          and p["file_url"].lower().endswith(supported_types)]

def download_images():
  data = get_json(url)
  count = data["@attributes"]["count"]

  if count == 0:
    print("📷 No results found")
    return

  print(f"🎯 Found {count} results")
  test_url = "https://gelbooru.com/index.php?page=post&s=list&tags={}".format(tags)
  display(Markdown(f"[Click here to open in browser!]({test_url})"))
  print (f"🔽 Will download to {images_folder.replace('/content/drive/', '')} (A confirmation box should appear below, otherwise run this cell again)")
  inp = input("❓ Enter the word 'yes' if you want to proceed with the download: ")

  if inp.lower().strip() != 'yes':
    print("❌ Download cancelled")
    return

  print("📩 Grabbing image list...")

  image_urls = set()
  image_urls = image_urls.union(filter_images(data))
  for i in range(total_limit // limit):
    count -= limit
    if count <= 0:
      break
    time.sleep(0.1)
    image_urls = image_urls.union(filter_images(get_json(url+f"&pid={i+1}")))

  scrape_file = os.path.join(config_folder, f"scrape_{project_subfolder}.txt")
  with open(scrape_file, "w") as f:
    f.write("\n".join(image_urls))

  print(f"🌐 Saved links to {scrape_file}\n\n🔁 Downloading images...\n")
  old_img_count = len([f for f in os.listdir(images_folder) if f.lower().endswith(supported_types)])

  os.chdir(images_folder)
  !aria2c --console-log-level=warn -c -x 16 -k 1M -s 16 -i {scrape_file}

  new_img_count = len([f for f in os.listdir(images_folder) if f.lower().endswith(supported_types)])
  print(f"\n✅ Downloaded {new_img_count - old_img_count} images.")

download_images()

### 2️⃣ 他のスクレイピング（自作）

In [ ]:
#@title kemono.party からスクレイピング

from bs4 import BeautifulSoup
import requests
import re

def get_html(url):
  response = requests.get(url)
  return response.content

def get_images(url):
  global file
  content = BeautifulSoup(get_html(url), 'html.parser')
  pattern = re.compile(r'.+c\d\.kemono\.su.+\.jpg')
  urls = content.find_all('a', href=pattern)
  urls = [link['href'] for link in urls]
  if urls:
    urls.pop(0)
    for img_url in urls:
      # file.write(img_url+"\n")
      !aria2c --console-log-level=warn -c -x 16 -k 1M -s 16 {img_url}

def main(url, count):
  for n in range(0, count*50, 50):
    content = BeautifulSoup(get_html(f"{url}?o={n}"), 'html.parser')
    pattern = re.compile(r'/fanbox/user/54262103/post/\w+')
    urls = content.find_all('a', href=pattern)
    urls = [link['href'] for link in urls]
    global no_url
    for sub_url in urls:
      if sub_url not in no_url:
        get_images("https://kemono.su" + sub_url)

url = "https://kemono.su/fanbox/user/54262103" # @param {type:"string"}
no_url = "" # @param {type:"string"}
no_url = no_url.split(", ")  # カンマとスペースで文字列を分割してリスト化
no_url = [string.replace('https://kemono.su', '') for string in no_url]
no_url = set(no_url)  # 重複を除去するためにセットに変換
no_url = list(no_url)  # セットをリストに変換
page_count = 4 # @param {type:"integer"}

# with open("/content/img_list.txt", "w") as file:
os.chdir(images_folder)
main(url, page_count)
print(f"You now have {len(os.listdir(images_folder))} images.")

In [ ]:
#@title Youtubeからダウンロード（不安定）

def ubuntu_deps():
  print("🏭 Installing dependencies...\n")
  !apt install yt-dlp
  return not get_ipython().__dict__['user_ns']['_exit_code']

if "yt_dlp_installed_flag" not in globals():
  if ubuntu_deps():
    yt_dlp_installed_flag = True
  else:
    print("❌ Error installing dependencies, attempting to continue anyway...")

os.makedir("/content/videos", exist_ok=True)
os.chdir("/content/videos")
youtubr_url = "" # @param {type:"string"}
!yt-dlp $youtube_url

In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

import json
import time
from urllib.request import urlopen, Request

#@title ### Gelbooruから動画をダウンロード

#@markdown We will grab images from the popular anime gallery [Gelbooru](https://gelbooru.com/). Images are sorted by tags, including poses, scenes, character traits, character names, artists, etc. <p>
tags = "animated score:>10000" #@param {type:"string"}
##@markdown If an image is bigger than this resolution a smaller version will be downloaded instead.
max_resolution = 3072 #@param {type:"slider", min:1024, max:8196, step:1024}
##@markdown Posts with a parent post are often minor variations of the same image.
include_posts_with_parent = True #param {type:"boolean"}

tags = tags.replace(" ", "+")\
           .replace("(", "%28")\
           .replace(")", "%29")\
           .replace(":", "%3a")\
           .replace("&", "%26")\

url = "https://gelbooru.com/index.php?page=dapi&json=1&s=post&q=index&limit=100&tags={}".format(tags)
user_agent = "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/93.0.4577.83 Safari/537.36"
limit = 100 # hardcoded by gelbooru
total_limit = 1000 # you can edit this if you want but I wouldn't recommend it
supported_types = (".webm", ".mp4", ".gif")

def get_json(url):
  with urlopen(Request(url, headers={"User-Agent": user_agent})) as page:
    return json.load(page)

def filter_images(data):
  return [p["file_url"] if p["width"]*p["height"] <= max_resolution**2 else p["sample_url"]
          for p in data["post"]
          if (p["parent_id"] == 0 or include_posts_with_parent)
          and p["file_url"].lower().endswith(supported_types)]

def download_images():
  videos_folder = "/content/videos"
  data = get_json(url)
  count = data["@attributes"]["count"]
  os.makedirs(videos_folder, exist_ok=True)

  if count == 0:
    print("📷 No results found")
    return

  print(f"🎯 Found {count} results")
  test_url = "https://gelbooru.com/index.php?page=post&s=list&tags={}".format(tags)
  display(Markdown(f"[Click here to open in browser!]({test_url})"))
  print (f"🔽 Will download to {videos_folder.replace('/content/drive/', '')} (A confirmation box should appear below, otherwise run this cell again)")
  inp = input("❓ Enter the word 'yes' if you want to proceed with the download: ")

  if inp.lower().strip() != 'yes':
    print("❌ Download cancelled")
    return

  print("📩 Grabbing image list...")

  image_urls = set()
  image_urls = image_urls.union(filter_images(data))
  for i in range(total_limit // limit):
    count -= limit
    if count <= 0:
      break
    time.sleep(0.1)
    image_urls = image_urls.union(filter_images(get_json(url+f"&pid={i+1}")))

  scrape_file = os.path.join(config_folder, f"scrape_{project_subfolder}.txt")
  with open(scrape_file, "w") as f:
    f.write("\n".join(image_urls))

  print(f"🌐 Saved links to {scrape_file}\n\n🔁 Downloading images...\n")
  old_img_count = len([f for f in os.listdir(videos_folder) if f.lower().endswith(supported_types)])

  os.chdir(videos_folder)
  !aria2c --console-log-level=warn -c -x 16 -k 1M -s 16 -i {scrape_file}

  new_img_count = len([f for f in os.listdir(videos_folder) if f.lower().endswith(supported_types)])
  print(f"\n✅ Downloaded {new_img_count - old_img_count} images.")

download_images()

In [ ]:
#@title videos.txtを作成

with open("/content/videos.txt", "w") as file:
  for i in range(1, 9, 1) :

    file.write(f"https://archive.org/download/do-not-do-it-self/Do%20It%20Yourself%21%21/Do%20It%20Yourself%21%21%20Ep%20{i}.mp4\n")


In [ ]:
#@title ### video.txtから動画をダウンロード

import cv2, os
from urllib.parse import unquote

#aria2cのセットアップ
def ubuntu_deps():
  print("🏭 Installing dependencies...\n")
  !apt -y install aria2
  return not get_ipython().__dict__['user_ns']['_exit_code']

if "step2_installed_flag" not in globals():
  if ubuntu_deps():
    step2_installed_flag = True
  else:
    print("❌ Error installing dependencies, attempting to continue anyway...")

def video_download():
  video_path = "/content/videos" # @param {type:"string"}
  os.makedirs(video_path, exist_ok=True)
  os.chdir(video_path)
  !aria2c -c --console-log-level=warn -x 16 -k 1M -s 16 -i "/content/videos.txt"
  num = 1
  for file in os.listdir(video_path):
      cap = cv2.VideoCapture(file)
      if cap :
        print(num, ":", file, "の合計フレーム数は：" , int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), "FPSは：" , round(cap.get(cv2.CAP_PROP_FPS)))
        num += 1

def save_frame(cap, name, start_frame, end_frame, step_frame, save_dir, ext='jpg'):
  digit = len(str(end_frame))
  for n in range(start_frame, end_frame, step_frame):
    cap.set(cv2.CAP_PROP_POS_FRAMES, n)
    ret, frame = cap.read()
    if ret:
        cv2.imwrite('{}/{}_{}.{}'.format(save_dir, name, str(n).zfill(digit), ext), frame)
    else:
        return
  print(name , "captured!!\n")

video_download()

In [ ]:
#@title ### キャプチャ
# @markdown 最終枚数は400～800枚 12話アニメだと60枚/話 567frame/stepくらい
import os, cv2

def save_frame(cap, name, start_frame, step_frame, save_dir, ext='jpg'):
  end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  for n in range(start_frame, end_frame, step_frame):
    cap.set(cv2.CAP_PROP_POS_FRAMES, n)
    ret, frame = cap.read()
    if ret:
        cv2.imwrite('{}/{}_{}.{}'.format(save_dir, name, str(n).zfill(len(str(end_frame))), ext), frame)
    else:
        return

video_path = "/content/videos/" # @param {type:"string"}
for file in os.listdir(video_path) :
  start_frame = 0 # @param {type:"integer"}
  frame_steps = 100 # @param {type:"integer"}
  cap = cv2.VideoCapture(video_path + file)
  if cap :
    save_frame(cap, file, start_frame, frame_steps, images_folder)
    print(file , "captured!!")
  else : print(file, "is not video.")
print(f"You now have {len(os.listdir(images_folder))} images.")

Do It Yourself!! Ep 12.mp4 captured!!
Do It Yourself!! Ep 5.mp4 captured!!
Do It Yourself!! Ep 8.mp4 captured!!
Do It Yourself!! Ep 1.mp4 captured!!
Do It Yourself!! Ep 6.mp4 captured!!
Do It Yourself!! Ep 2.mp4 captured!!
Do It Yourself!! Ep 7.mp4 captured!!
Do It Yourself!! Ep 11.mp4 captured!!
Do It Yourself!! Ep 9.mp4 captured!!
Do It Yourself!! Ep 4.mp4 captured!!
Do It Yourself!! Ep 10.mp4 captured!!
Do It Yourself!! Ep 3.mp4 captured!!
You now have 4092 images.


### 3️⃣ Curate your images

In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@title 3️⃣ Curate your images
#@markdown We will find duplicate images with the FiftyOne AI, and mark them with `delete`. <p>
#@markdown Then, an interactive area will appear below this cell that lets you visualize all your images and manually mark with `delete` to the ones you don't like. <p>
#@markdown If the interactive area appears blank for over a minute, try enabling cookies and removing tracking protection for the Google Colab website, as they may break it.
#@markdown Regardless, you can save your changes by sending Enter in the input box above the interactive area.<p>
#@markdown This is how similar 2 images must be to be marked for deletion. I recommend 0.97 to 0.99:
similarity_threshold = 0.97 #@param {type:"number"}


os.chdir(root_dir)
model_name = "clip-vit-base32-torch"
supported_types = (".png", ".jpg", ".jpeg")
img_count = len(os.listdir(images_folder))
batch_size = min(250, img_count)

if "step3_installed_flag" not in globals():
  print("🏭 Installing dependencies...\n")
  !pip -q install fiftyone ftfy
  !pip -q install fiftyone-db-ubuntu2204
  if not get_ipython().__dict__['user_ns']['_exit_code']:
    clear_output()
    step3_installed_flag = True
  else:
    print("❌ Error installing dependencies, attempting to continue anyway...")

import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
from sklearn.metrics.pairwise import cosine_similarity

non_images = [f for f in os.listdir(images_folder) if not f.lower().endswith(supported_types)]
if non_images:
  print(f"💥 Error: Found non-image file {non_images[0]} - This program doesn't allow it. Sorry! Use the Extras at the bottom to clean the folder.")
elif img_count == 0:
  print(f"💥 Error: No images found in {images_folder}")
else:
  print("\n💿 Analyzing dataset...\n")
  dataset = fo.Dataset.from_dir(images_folder, dataset_type=fo.types.ImageDirectory)
  model = foz.load_zoo_model(model_name)
  embeddings = dataset.compute_embeddings(model, batch_size=batch_size)

  batch_embeddings = np.array_split(embeddings, batch_size)
  similarity_matrices = []
  max_size_x = max(array.shape[0] for array in batch_embeddings)
  max_size_y = max(array.shape[1] for array in batch_embeddings)

  for i, batch_embedding in enumerate(batch_embeddings):
    similarity = cosine_similarity(batch_embedding)
    #Pad 0 for np.concatenate
    padded_array = np.zeros((max_size_x, max_size_y))
    padded_array[0:similarity.shape[0], 0:similarity.shape[1]] = similarity
    similarity_matrices.append(padded_array)

  similarity_matrix = np.concatenate(similarity_matrices, axis=0)
  similarity_matrix = similarity_matrix[0:embeddings.shape[0], 0:embeddings.shape[0]]

  similarity_matrix = cosine_similarity(embeddings)
  similarity_matrix -= np.identity(len(similarity_matrix))

  dataset.match(F("max_similarity") > similarity_threshold)
  dataset.tags = ["delete", "has_duplicates"]

  id_map = [s.id for s in dataset.select_fields(["id"])]
  samples_to_remove = set()
  samples_to_keep = set()

  for idx, sample in enumerate(dataset):
    if sample.id not in samples_to_remove:
      # Keep the first instance of two duplicates
      samples_to_keep.add(sample.id)

      dup_idxs = np.where(similarity_matrix[idx] > similarity_threshold)[0]
      for dup in dup_idxs:
          # We kept the first instance so remove all other duplicates
          samples_to_remove.add(id_map[dup])

      if len(dup_idxs) > 0:
          sample.tags.append("has_duplicates")
          sample.save()
    else:
      sample.tags.append("delete")
      sample.save()

  clear_output()

  sidebar_groups = fo.DatasetAppConfig.default_sidebar_groups(dataset)
  for group in sidebar_groups[1:]:
    group.expanded = False
  dataset.app_config.sidebar_groups = sidebar_groups
  dataset.save()
  session = fo.launch_app(dataset)

  print("❗ Wait a minute for the session to load. If it doesn't, read above.")
  print("❗ When it's ready, you'll see a grid of your images.")
  print("❗ On the left side enable \"sample tags\" to visualize the images marked for deletion.")
  print("❗ You can mark your own images with the \"delete\" label by selecting them and pressing the tag icon at the top.")
  input("⭕ When you're done, enter something here to save your changes: ")

  print("💾 Saving...")

In [ ]:
kys = [s for s in dataset if "delete" in s.tags]
dataset.remove_samples(kys)
previous_folder = images_folder[:images_folder.rfind("/")]
dataset.export(export_dir=os.path.join(images_folder, project_subfolder), dataset_type=fo.types.ImageDirectory)

temp_suffix = "_temp"
!mv {images_folder} {images_folder}{temp_suffix}
!mv {images_folder}{temp_suffix}/{project_subfolder} {images_folder}
!rm -r {images_folder}{temp_suffix}

session.refresh()
fo.close_app()
clear_output()

print(f"\n✅ Removed {len(kys)} images from dataset. You now have {len(os.listdir(images_folder))} images.")


✅ Removed 178 images from dataset. You now have 921 images.


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@title ### 4️⃣ Tag your images
#@markdown We will be using AI to automatically tag your images, specifically [Waifu Diffusion](https://huggingface.co/SmilingWolf/wd-v1-4-swinv2-tagger-v2) in the case of anime and [BLIP](https://huggingface.co/spaces/Salesforce/BLIP) in the case of photos.
#@markdown Giving tags/captions to your images allows for much better training. This process should take a couple minutes. <p>
method = "Anime tags" #@param ["Anime tags", "Photo captions"]
#@markdown **Anime:** The threshold is the minimum level of confidence the tagger must have in order to include a tag. Lower threshold = More tags. Recommended 0.35 to 0.5
tag_threshold = 0.35 #@param {type:"slider", min:0.0, max:1.0, step:0.01}
blacklist_tags = "bangs, breasts, multicolored hair, two-tone hair, gradient hair, virtual youtuber, official alternate costume, official alternate hairstyle, official alternate hair length, alternate costume, alternate hairstyle, alternate hair length, alternate hair color" #@param {type:"string"}
#@markdown `bangs, breasts, multicolored hair, two-tone hair, gradient hair, virtual youtuber, official alternate costume, official alternate hairstyle, official alternate hair length, alternate costume, alternate hairstyle, alternate hair length, alternate hair color` <p>
#@markdown **Photos:** The minimum and maximum length of tokens/words in each caption.
caption_min = 10 #@param {type:"number"}
caption_max = 75 #@param {type:"number"}

%env PYTHONPATH=/env/python
os.chdir(root_dir)
kohya = "/content/kohya-trainer"
if not os.path.exists(kohya):
  !git clone https://github.com/kohya-ss/sd-scripts {kohya}
  os.chdir(kohya)
  !git reset --hard 5050971ac687dca70ba0486a583d283e8ae324e2
  os.chdir(root_dir)

if "tags" in method:
  if "step4a_installed_flag" not in globals():
    print("\n🏭 Installing dependencies...\n")
    !pip -q install tensorflow==2.12.0 huggingface-hub==0.12.0 accelerate==0.15.0 transformers==4.26.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6 torchvision albumentations
    if not get_ipython().__dict__['user_ns']['_exit_code']:
      clear_output()
      step4a_installed_flag = True
    else:
      print("❌ Error installing dependencies, trying to continue anyway...")

  print("\n🚶‍♂️ Launching program...\n")

  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
  %env PYTHONPATH={kohya}
  !python {kohya}/finetune/tag_images_by_wd14_tagger.py \
    {images_folder} \
    --repo_id=SmilingWolf/wd-v1-4-swinv2-tagger-v2 \
    --model_dir={root_dir} \
    --thresh={tag_threshold} \
    --batch_size=8 \
    --caption_extension=.txt \
    --force_download

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    print("removing underscores and blacklist...")
    blacklisted_tags = [t.strip() for t in blacklist_tags.split(",")]
    from collections import Counter
    top_tags = Counter()
    for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
      with open(os.path.join(images_folder, txt), 'r') as f:
        tags = [t.strip() for t in f.read().split(",")]
        tags = [t.replace("_", " ") if len(t) > 3 else t for t in tags]
        tags = [t for t in tags if t not in blacklisted_tags]
      top_tags.update(tags)
      with open(os.path.join(images_folder, txt), 'w') as f:
        f.write(", ".join(tags))

    %env PYTHONPATH=/env/python
    clear_output()
    print(f"📊 Tagging complete. Here are the top 50 tags in your dataset:")
    print("\n".join(f"{k} ({v})" for k, v in top_tags.most_common(50)))


else: # Photos
  if "step4b_installed_flag" not in globals():
    print("\n🏭 Installing dependencies...\n")
    !pip -q install timm==0.6.12 fairscale==0.4.13 transformers==4.26.0 requests==2.28.2 accelerate==0.15.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6
    if not get_ipython().__dict__['user_ns']['_exit_code']:
      clear_output()
      step4b_installed_flag = True
    else:
      print("❌ Error installing dependencies, trying to continue anyway...")

  print("\n🚶‍♂️ Launching program...\n")

  os.chdir(kohya)
  %env PYTHONPATH={kohya}
  !python {kohya}/finetune/make_captions.py \
    {images_folder} \
    --beam_search \
    --max_data_loader_n_workers=2 \
    --batch_size=8 \
    --min_length={caption_min} \
    --max_length={caption_max} \
    --caption_extension=.txt

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    import random
    captions = [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]
    sample = []
    for txt in random.sample(captions, min(10, len(captions))):
      with open(os.path.join(images_folder, txt), 'r') as f:
        sample.append(f.read())

    os.chdir(root_dir)
    %env PYTHONPATH=/env/python
    clear_output()
    print(f"📊 Captioning complete. Here are {len(sample)} example captions from your dataset:")
    print("".join(sample))

📊 Tagging complete. Here are the top 50 tags in your dataset:
1girl (902)
blonde hair (856)
blush (778)
long hair (727)
braid (606)
nipples (569)
yellow eyes (557)
hat (531)
witch hat (523)
pussy (505)
solo (489)
censored (445)
open mouth (439)
hetero (396)
1boy (386)
bow (369)
solo focus (338)
nude (336)
single braid (333)
penis (331)
navel (305)
cum (253)
spread legs (248)
hair bow (236)
sex (226)
small breasts (223)
vaginal (222)
tears (212)
underwear (203)
looking at viewer (201)
medium breasts (193)
bar censor (176)
side braid (175)
lying (172)
simple background (171)
large breasts (166)
pussy juice (158)
sweat (156)
panties (150)
ass (150)
clothes lift (148)
hair between eyes (139)
thighhighs (136)
anus (135)
heart (130)
black headwear (128)
on back (127)
shirt (126)
uncensored (126)
cum in pussy (125)


In [ ]:
#@title 5️⃣ 指定のタグが入った画像を削除（自作）
#@markdown 指定タグ以外の画像を消す場合 nor_delete にチェック
nor_delete = False # @param {type:"boolean"}
#@markdown タグは1つだけ
target_tag = "boy" # @param {type:"string"}
import os

def func(path):
  if os.path.isfile(path) :
    print(os.path.basename(path))
    os.remove(path)
    #os.rename(path, "../sub" + os.path.basename(path))
  else : return

def main(directory, tag, nor):
  for filename in os.listdir(directory) :
    if filename.endswith(".txt") :
      filename = os.path.join(directory, filename)
      with open(filename, "r") as file:
        filename = os.path.splitext(filename)[0]
        content = file.read()
        if tag in content and not nor:
          func(filename + ".txt")
          func(filename + ".jpg")
          func(filename + ".png")
          func(filename + ".jpeg")
        if tag not in content and nor:
          func(filename + ".txt")
          func(filename + ".jpg")
          func(filename + ".png")
          func(filename + ".jpeg")

os.chdir(config_folder)
#!mkdir sub
main(images_folder, target_tag, nor_delete)
print(f"You now have {len(os.listdir(images_folder))/2} images.")

In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@title ### 6️⃣ Curate your tags
#@markdown Modify your dataset's tags. You can run this cell multiple times with different parameters. <p>

#@markdown Put an activation tag at the start of every text file. This is useful to make learning better and activate your Lora easier. Set `keep_tokens` to 1 when training.<p>
#@markdown Common tags that are removed such as hair color, etc. will be "absorbed" by your activation tag.
global_activation_tag = "krsm mrs" #@param {type:"string"}
remove_tags = "blonde hair, long hair, braid, yellow eyes, single braid, hair between eyes, " #@param {type:"string"}
#@markdown &nbsp;

#@markdown In this advanced section, you can search text files containing matching tags, and replace them with less/more/different tags. If you select the checkbox below, any extra tags will be put at the start of the file, letting you assign different activation tags to different parts of your dataset. Still, you may want a more advanced tool for this.
search_tags = "" #@param {type:"string"}
replace_with = "" #@param {type:"string"}
search_mode = "OR" #@param ["OR", "AND"]
new_becomes_activation_tag = False #@param {type:"boolean"}
#@markdown These may be useful sometimes. Will remove existing activation tags, be careful.
sort_alphabetically = False #@param {type:"boolean"}
remove_duplicates = False #@param {type:"boolean"}

def split_tags(tagstr):
  return [s.strip() for s in tagstr.split(",") if s.strip()]

activation_tag_list = split_tags(global_activation_tag)
remove_tags_list = split_tags(remove_tags)
search_tags_list = split_tags(search_tags)
replace_with_list = split_tags(replace_with)
replace_new_list = [t for t in replace_with_list if t not in search_tags_list]

replace_with_list = [t for t in replace_with_list if t not in replace_new_list]
replace_new_list.reverse()
activation_tag_list.reverse()

remove_count = 0
replace_count = 0

for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:

  with open(os.path.join(images_folder, txt), 'r') as f:
    tags = [s.strip() for s in f.read().split(",")]

  if remove_duplicates:
    tags = list(set(tags))
  if sort_alphabetically:
    tags.sort()

  for rem in remove_tags_list:
    if rem in tags:
      remove_count += 1
      tags.remove(rem)

  if "AND" in search_mode and all(r in tags for r in search_tags_list) \
      or "OR" in search_mode and any(r in tags for r in search_tags_list):
    replace_count += 1
    for rem in search_tags_list:
      if rem in tags:
        tags.remove(rem)
    for add in replace_with_list:
      if add not in tags:
        tags.append(add)
    for new in replace_new_list:
      if new_becomes_activation_tag:
        if new in tags:
          tags.remove(new)
        tags.insert(0, new)
      else:
        if new not in tags:
          tags.append(new)

  for act in activation_tag_list:
    if act in tags:
      tags.remove(act)
    tags.insert(0, act)

  with open(os.path.join(images_folder, txt), 'w') as f:
    f.write(", ".join(tags))

if global_activation_tag:
  print(f"\n📎 Applied new activation tag(s): {', '.join(activation_tag_list)}")
if remove_tags:
  print(f"\n🚮 Removed {remove_count} tags.")
if search_tags:
  print(f"\n💫 Replaced in {replace_count} files.")
print("\n✅ Done! Check your updated tags in the Extras below.")


📎 Applied new activation tag(s): krsm mrs

🚮 Removed 3218 tags.

✅ Done! Check your updated tags in the Extras below.


In [ ]:
#@title 7️⃣ README.txt を作成（自作）
# @markdown 備考欄を入力　\nで改行
import datetime, os

def create_txt():
  with open(f"{config_folder}/README.txt", "w") as file:
    file.write(f"# {project_name}\n")
    file.write(f"Made at {datetime.date.today()}.</b> <p>")
    file.write(f"Datasets have {len(os.listdir(images_folder))/2} images.</b> <p>")
    text = "\u9727\u96E8\u9B54\u7406\u6C99\u306EExplicit\u3092Gelbooru\u3067\u30B9\u30AF\u30EC\u30A4\u30D7  \\n\u30C8\u30EA\u30AC\u30FC\u306Fkrsm mrs\u306B\u3057\u3066\u3044\u308B" # @param {type:"string"}
    text = text.replace('\\n','  \n')
    file.write(f"{text}  \n")
    if "start_frame" in globals():
      file.write(f"\n<b>This lora was trained from video.</b>  \n")
      file.write(f"start_frame: {start_frame}  \n")
      file.write(f"frame_steps: {frame_steps}  \n")
    if "scrape_tags" in globals():
      file.write(f"\n<b>This lora was trained from gelbooru scraiping.</b>  \n")
      file.write(f"scrape_tags: {scrape_tags}  \n")
    if "target_tag" in globals():
      if nor_delete : file.write(f"\nDeleted images that haven't target_tag.</b>  \n")
      else : file.write(f"\n<b>Deleted images that have target_tag.</b>  \n")
      file.write(f"target_tag: {target_tag}  \n")
    if "global_activation_tag" in globals():
      file.write(f"\n<b>Tags fixed.</b>  \n")
      file.write(f"global_activation_tag: {global_activation_tag}  \n")
      if remove_tags: file.write(f"remove_tags: {remove_tags}  \n")

create_txt()

## *️⃣ Extras 0️⃣1️⃣2️⃣3️⃣4️⃣6️⃣7️⃣8️⃣9️⃣

In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@title ### 📈 Analyze Tags
#@markdown Perhaps you need another look at your dataset.
show_top_tags = 50 #@param {type:"number"}

from collections import Counter
top_tags = Counter()

for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
  with open(os.path.join(images_folder, txt), 'r') as f:
    top_tags.update([s.strip() for s in f.read().split(",")])

top_tags = Counter(top_tags)
print(f"📊 Top {show_top_tags} tags:")
for k, v in top_tags.most_common(show_top_tags):
  print(f"{k} ({v})")

📊 Top 50 tags:
krsm mrs (921)
1girl (902)
blush (778)
nipples (569)
hat (531)
witch hat (523)
pussy (505)
solo (489)
censored (445)
open mouth (439)
hetero (396)
1boy (386)
bow (369)
solo focus (338)
nude (336)
penis (331)
navel (305)
cum (253)
spread legs (248)
hair bow (236)
sex (226)
small breasts (223)
vaginal (222)
tears (212)
underwear (203)
looking at viewer (201)
medium breasts (193)
bar censor (176)
side braid (175)
lying (172)
simple background (171)
large breasts (166)
pussy juice (158)
sweat (156)
panties (150)
ass (150)
clothes lift (148)
thighhighs (136)
anus (135)
heart (130)
black headwear (128)
on back (127)
shirt (126)
uncensored (126)
cum in pussy (125)
skirt (120)
bottomless (120)
smile (116)
socks (113)
white background (111)


In [ ]:
#@markdown ### 📂 Unzip dataset
#@markdown It's much slower to upload individual files to your Drive, so you may want to upload a zip if you have your dataset in your computer.
zip = "/content/drive/MyDrive/Loras/example.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/example/dataset" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Done")

In [ ]:
#@markdown ### 🔢 Count datasets
#@markdown Google Drive makes it impossible to count the files in a folder, so this will show you the file counts in all folders and subfolders.
folder = "/content/drive/MyDrive/Loras" #@param {type:"string"}

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Connecting to Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
  dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images else f"{images:>4} images | {captions:>4} captions |"
  if tree[path] and others:
    tree[path] += f" {others:>4} other files"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))

In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@title ### 🚮 Clean folder
#@markdown Careful! Deletes all non-image files in the project folder.

!find {images_folder} -type f ! \( -name '*.png' -o -name '*.jpg' -o -name '*.jpeg' \) -delete
